<a href="https://colab.research.google.com/github/nanakoKuze/Deep-Learning/blob/main/p3_2025_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW3: Classification
In this exercise, you will fine-tune a pre-trained convolutional neural network (CNN) for image classification. The dataset contains chess positions labeled as either 'insufficient material' or 'sufficient material.'
Make sure to upload the provided dataset to Google Colab before running the code.

## Package Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image
!unzip /chess_dataset.zip
torch.manual_seed(0)

Archive:  /chess_dataset.zip
   creating: chess_dataset/
  inflating: __MACOSX/._chess_dataset  
  inflating: chess_dataset/.DS_Store  
  inflating: __MACOSX/chess_dataset/._.DS_Store  
   creating: chess_dataset/train/
  inflating: __MACOSX/chess_dataset/._train  
   creating: chess_dataset/val/
  inflating: __MACOSX/chess_dataset/._val  
   creating: chess_dataset/train/sufficient_material/
  inflating: __MACOSX/chess_dataset/train/._sufficient_material  
   creating: chess_dataset/train/insufficient_material/
  inflating: __MACOSX/chess_dataset/train/._insufficient_material  
   creating: chess_dataset/val/sufficient_material/
  inflating: __MACOSX/chess_dataset/val/._sufficient_material  
   creating: chess_dataset/val/insufficient_material/
  inflating: __MACOSX/chess_dataset/val/._insufficient_material  
  inflating: chess_dataset/train/sufficient_material/sufficient-122.png  
  inflating: __MACOSX/chess_dataset/train/sufficient_material/._sufficient-122.png  
  inflating: chess_

## Dataset

In [ ]:
# Define chess dataset
class ChessDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory = directory
        self.transform = transform
        self.images = []
        self.labels = []

        for label, subdir in enumerate(['insufficient_material', 'sufficient_material']):
            subdir_path = os.path.join(directory, subdir)
            for filename in os.listdir(subdir_path):
                if filename.endswith('.png'):
                    self.images.append(os.path.join(subdir_path, filename))
                    self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

## Part (a): Model and Training Setup

In [ ]:
# load a pretrained ResNet model
model = models.resnet18(weights='ResNet18_Weights.DEFAULT')

# create the dataset
# Make sure the 'chess_dataset' folder is correctly uploaded to Google Colab and update the path if needed
train_dataset = ChessDataset('chess_dataset/train', transform=data_transforms['train']) # put the path to your chess_dataset folder!
val_dataset = ChessDataset('chess_dataset/val', transform=data_transforms['val'])

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

##########################
#  Beginning of Part a  ##
##########################

# TODO: modify the fully connected layer for binary classification
# hint: it might be helpful to print(model) to see how the model is structured

# use a GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

# create the dataloader
# hint: remember to shuffle the train_loader
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

## Part (b): Train and Validation

In [ ]:
###########################
##  Beginning of Part b  ##
###########################

def train(epoch_num):
    # set the model to train mode
    model.train()

    running_loss = 0.0
    total_loss = 0.0
    running_count = 0
    total_count = 0

    for batch_index, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        '''
        TODO: complete the training loop. You will need to do the following:
        1. zero the parameter gradients
        2. run a forward pass
        3. run a backwards pass and optimizer step
        '''
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        total_loss += loss.item() * inputs.size(0)
        running_count += inputs.size(0)
        total_count += inputs.size(0)

        # update loss and count
        running_loss += loss.item() * inputs.size(0)
        total_loss += loss.item() * inputs.size(0)

        running_count += inputs.size(0)
        total_count += inputs.size(0)

        # print every 50 mini-batches
        if batch_index % 50 == 49:
            print('[%d, %5d] avg batch loss: %.3f avg epoch loss: %.3f' %
                (epoch_num + 1, batch_index + 1, running_loss / running_count, total_loss / total_count))
            running_loss = 0.0
            running_count = 0


def validate():
    # set the model to evaluation mode
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_count = 0

    # Validation loop
    with torch.no_grad():  # No need to track gradients for validation
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            '''
            TODO: run the forward pass (same as previous part)
            from your output (which are probabilities for each class, find the predicted class)
            '''
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            predicted = torch.max(outputs, 1)[1]

            correct_count = (predicted == labels).sum().item()

            # update loss and count
            total_loss += loss.item() * labels.size(0)
            total_correct += correct_count
            total_count += labels.size(0)

    accuracy = 100 * total_correct / total_count
    print()
    print(f"Evaluation loss: {total_loss / total_count :.3f}")
    print(f'Accuracy of the model on the validation images: {accuracy: .2f}%')
    print()

## Part (c)

In [ ]:
###########################
##  Beginning of Part c  ##
###########################

# TODO: validate your model before training
validate()

num_epochs = 1
# TODO: train and validate your model

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # Train the model
    train(epoch)

    # Validate after each epoch
    validate()


Evaluation loss: 1.499
Accuracy of the model on the validation images:  40.00%

Epoch 1/1
[1,    50] avg batch loss: 0.836 avg epoch loss: 0.836
[1,   100] avg batch loss: 0.433 avg epoch loss: 0.635
[1,   150] avg batch loss: 0.263 avg epoch loss: 0.511
[1,   200] avg batch loss: 0.169 avg epoch loss: 0.425
[1,   250] avg batch loss: 0.150 avg epoch loss: 0.370
Evaluating model after training...

Evaluation loss: 0.030
Accuracy of the model on the validation images:  100.00%

